In [80]:
#wikipedia link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')


In [81]:
table_contents=[]
table=soup.findAll('table')[0]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Create a dataframe from List & Cleaning up Borough column

In [82]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Looking at all rows to make sure data looks clean (esp. the Borough)

In [83]:
#toggle this on/off to see all rows
#pd.set_option("display.max_rows",  None)

df.head() #only see the first 5


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [84]:
df.shape

(103, 3)

Adding Longitude and Latitude

In [86]:
#Getting the files from csv file in my github since geocoder was unreliable
url2 = "https://raw.githubusercontent.com/aymiee/Toronto/master/Geospatial_Coordinates.csv"
#s=requests.get(url2).content
dfCoordinates = pd.read_csv(url2)
#print(dfCoordinates.loc[dfCoordinates['Postal Code'] == 'M3A'].Latitude)

df['Latitude'] = None
df['Longitude'] = None

for index, row in df.iterrows():  
    Latitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Latitude
    Longitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Latitude
    row['Latitude'] = float(Latitude)
    row['Longitude'] = float(Longitude)
    
df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,43.7533
1,M4A,North York,Victoria Village,43.7259,43.7259
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,43.6543
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,43.7185
4,M7A,Queen's Park,Ontario Provincial Government,43.6623,43.6623
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,43.6537
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,43.6659
100,M7Y,East Toronto Business,Enclave of M4L,43.6627,43.6627
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,43.6363
